In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import sqlite3
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
from nba_api.stats.endpoints import leaguegamelog, playergamelogs, teamgamelogs
from time import sleep
import time

In [2]:
def load_team_data(connection, start_season, end_season):
    """Loads basic, advanced, and scoring boxscores 
    from sqlite db and merges them into one dataframe"""
    

    df = pd.read_sql("""SELECT b.*, E_OFF_RATING, E_DEF_RATING, E_NET_RATING, AST_PCT, AST_TOV, AST_RATIO, OREB_PCT, DREB_PCT, REB_PCT,
        E_TM_TOV_PCT, TM_TOV_PCT, EFG_PCT, TS_PCT, E_USG_PCT, E_PACE, POSS, PIE, PCT_FGA_2PT, PCT_FGA_3PT, PCT_PTS_2PT, PCT_PTS_2PT_MR, 
        PCT_PTS_3PT, PCT_AST_2PM, PCT_UAST_2PM, PCT_AST_3PM, PCT_UAST_3PM, 
        PCT_AST_FGM, PCT_UAST_FGM, contestedShots, contestedShots2pt, contestedShots3pt, deflections, chargesDrawn, screenAssists,
        screenAssistPoints, looseBallsRecoveredOffensive, looseBallsRecoveredDefensive, looseBallsRecoveredTotal, offensiveBoxOuts,
        defensiveBoxOuts, boxOutPlayerTeamRebounds, boxOutPlayerRebounds, boxOuts, DIST, ORBC, DRBC, RBC, TCHS, SAST, FTAST, PASS,
        CFGM, CFGA, CFG_PCT, UFGM, UFGA, UFG_PCT, DFGM, DFGA, DFG_PCT, PTS_OFF_TOV, PTS_2ND_CHANCE, PTS_FB, PTS_PAINT
        FROM team_basic_stats b 
        LEFT JOIN team_advanced_stats adv ON b.team_ID = adv.team_ID AND b.GAME_ID = adv.GAME_ID
        LEFT JOIN team_scoring_stats sco ON b.team_ID = sco.team_ID AND b.GAME_ID = sco.GAME_ID 
        LEFT JOIN team_hustle_stats hust ON b.team_ID = hust.teamId AND b.GAME_ID = hust.gameId
        LEFT JOIN team_track_stats track ON b.team_ID = track.team_ID AND b.GAME_ID = track.GAME_ID
        LEFT JOIN team_miscellaneous_stats misc ON b.team_ID = misc.team_ID AND b.GAME_ID = misc.GAME_ID
                                """, connection)
    
    df = df.loc[df['SEASON_YEAR'].between(start_season, end_season)]
    
    return df


start_season = '2016-17'
end_season = '2023-24'

connection = sqlite3.connect('nba_stats.db')
df = load_team_data(connection, start_season, end_season)
connection.close()

In [65]:
def clean_team_data(df):
    """This function cleans the team_data
    1) Changes W/L to 1/0 
    2) Changes franchise abbreviations to their most 
    recent abbreviation for consistency
    3) Converts GAME_DATE to datetime object
    4) Creates a binary column 'HOME_GAME'
    5) Removes 3 games where advanced stats were not collected
    """
    df = df.copy()
    df['WL'] = (df['WL'] == 'W').astype(int)
    df = df.rename(columns={'SEASON_YEAR': 'SEASON'})

    abbr_mapping = {'NJN': 'BKN',
                    'CHH': 'CHA',
                    'VAN': 'MEM',
                    'NOH': 'NOP',
                    'NOK': 'NOP',
                    'SEA': 'OKC'}

    df['TEAM_ABBREVIATION'] = df['TEAM_ABBREVIATION'].replace(abbr_mapping)
    df['MATCHUP'] = df['MATCHUP'].str.replace('NJN', 'BKN')
    df['MATCHUP'] = df['MATCHUP'].str.replace('CHH', 'CHA')
    df['MATCHUP'] = df['MATCHUP'].str.replace('VAN', 'MEM')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOH', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOK', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('SEA', 'OKC')

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

    df['HOME_GAME'] = df['MATCHUP'].str.contains('vs').astype(int)

    return df


clean_df = clean_team_data(df)
clean_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,E_USG_PCT,E_PACE,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,contestedShots,contestedShots2pt,contestedShots3pt,deflections,chargesDrawn,screenAssists,screenAssistPoints,looseBallsRecoveredOffensive,looseBallsRecoveredDefensive,looseBallsRecoveredTotal,offensiveBoxOuts,defensiveBoxOuts,boxOutPlayerTeamRebounds,boxOutPlayerRebounds,boxOuts,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,DFGM,DFGA,DFG_PCT,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,HOME_GAME
2632,22016,1610612739,CLE,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,1,240,45,94,0.479,13,35,0.371,14,19,0.737,11,40,51,31,12,5,15,22,117,29,2016-17,110.0,87.3,22.7,0.689,2.07,20.9,0.353,0.729,0.555,14.103,14.9,0.548,0.572,0.198,103.58,101,0.703,0.628,0.372,0.547,0.103,0.333,0.625,0.375,0.846,0.154,0.689,0.311,65.0,46.0,19.0,16.0,0.0,6.0,12.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,17.12,41.0,72.0,108.0,441.0,2.0,0.0,314.0,17.0,39.0,0.436,28.0,55.0,0.509,10.0,18.0,0.556,21.0,16.0,16.0,52.0,1
2633,22016,1610612752,NYK,New York Knicks,0021600001,2016-10-25,NYK @ CLE,0,240,32,87,0.368,9,27,0.333,15,20,0.750,13,29,42,17,6,6,18,22,88,-29,2016-17,87.3,110.0,-22.7,0.531,0.94,13.0,0.271,0.647,0.445,17.857,17.8,0.420,0.459,0.202,103.58,101,0.297,0.690,0.310,0.523,0.205,0.307,0.391,0.609,0.889,0.111,0.531,0.469,65.0,39.0,26.0,9.0,2.0,7.0,15.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,17.29,34.0,71.0,104.0,408.0,1.0,1.0,287.0,12.0,31.0,0.387,20.0,56.0,0.357,21.0,31.0,0.677,14.0,16.0,9.0,28.0,0
2634,22016,1610612757,POR,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,1,240,39,75,0.520,13,19,0.684,22,22,1.000,5,29,34,22,5,3,13,18,113,9,2016-17,121.9,107.2,14.8,0.564,1.69,18.4,0.222,0.738,0.500,14.027,14.1,0.607,0.667,0.197,94.86,92,0.548,0.747,0.253,0.460,0.159,0.345,0.500,0.500,0.692,0.308,0.564,0.436,62.0,44.0,18.0,9.0,0.0,18.0,43.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,16.65,19.0,54.0,73.0,362.0,0.0,0.0,255.0,19.0,43.0,0.442,20.0,32.0,0.625,14.0,20.0,0.700,20.0,15.0,4.0,34.0,1
2635,22016,1610612762,UTA,Utah Jazz,0021600002,2016-10-25,UTA @ POR,0,240,40,82,0.488,8,24,0.333,16,16,1.000,6,25,31,19,9,5,14,19,104,-9,2016-17,107.2,121.9,-14.8,0.475,1.36,15.6,0.262,0.778,0.500,14.427,15.4,0.537,0.584,0.190,94.86,91,0.452,0.707,0.293,0.615,0.135,0.231,0.344,0.656,1.000,0.000,0.475,0.525,54.0,44.0,10.0,16.0,0.0,10.0,20.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,16.76,19.0,47.0,66.0,425.0,2.0,1.0,310.0,22.0,44.0,0.500,18.0,38.0,0.474,11.0,18.0,0.611,8.0,4.0,8.0,50.0,0
2636,22016,1610612744,GSW,Golden State Warriors,0021600003,2016-10-25,GSW vs. SAS,0,240,40,85,0.471,7,33,0.212,13,18,0.722,8,27,35,24,11,6,16,19,100,-29,2016-17,99.1,125.9,-26.8,0.600,1.50,18.1,0.208,0.528,0.376,15.854,15.8,0.512,0.538,0.198,101.68,101,0.400,0.612,0.388,0.660,0.180,0.210,0.576,0.424,0.714,0.286,0.600,0.400,83.0,62.0,21.0,23.0,0.0,5.0,11.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,16.90,23.0,45.0,67.0,397.0,4.0,1.0,277.0,21.0,38.0,0.553,19.0,47.0,0.404,20.0,32.0,0.625,15.0,4.0,20.0,48.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23043,42023,1610612738,BOS,Boston Celtics,0042300403,2024-06-12,BOS @ DAL,1,240,38,82,0.463,17,46,0.370,13,14,0.929,6,30,36,26,4,6,9,19,106,7,2023-24,116.3,104.2,12.1,0.684,2.89,21.1,0.136,0.750,0.457,9.873,9.8,0.567,0.601,0.200,93.10,92,0.544

In [66]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
clean_df.isnull().sum()

SEASON_ID                         0
TEAM_ID                           0
TEAM_ABBREVIATION                 0
TEAM_NAME                         0
GAME_ID                           0
GAME_DATE                         0
MATCHUP                           0
WL                                0
MIN                               0
FGM                               0
FGA                               0
FG_PCT                            0
FG3M                              0
FG3A                              0
FG3_PCT                           0
FTM                               0
FTA                               0
FT_PCT                            1
OREB                              0
DREB                              0
REB                               0
AST                               0
STL                               0
BLK                               0
TOV                               0
PF                                0
PTS                               0
PLUS_MINUS                  

In [67]:
df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'SEASON_YEAR',
       'E_OFF_RATING', 'E_DEF_RATING', 'E_NET_RATING', 'AST_PCT', 'AST_TOV',
       'AST_RATIO', 'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT',
       'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'E_USG_PCT', 'E_PACE', 'POSS', 'PIE',
       'PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_2PT', 'PCT_PTS_2PT_MR',
       'PCT_PTS_3PT', 'PCT_AST_2PM', 'PCT_UAST_2PM', 'PCT_AST_3PM',
       'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM', 'contestedShots',
       'contestedShots2pt', 'contestedShots3pt', 'deflections', 'chargesDrawn',
       'screenAssists', 'screenAssistPoints', 'looseBallsRecoveredOffensive',
       'looseBallsRecoveredDefensive', 'looseBallsRecoveredTotal',
       'offensiveBoxOuts', 'defensi

In [198]:
def prep_for_aggregation(df):
    """This function...
    1) Removes categories that are percentages,
    as we will be averaging them and do not want to average 
    percentages. 
    2) Converts shooting percentage stats into raw values"""
    df = df.copy()

    df['FG2M'] = df['FGM'] - df['FG3M']
    df['FG2A'] = df['FGA'] - df['FG3A']
    df['PTS_2PT_MR'] = (df['PTS'] * df['PCT_PTS_2PT_MR']).astype('int8')
    df['AST_2PM'] = (df['FG2M'] * df['PCT_AST_2PM']).astype('int8')
    df['AST_3PM'] = (df['FG3M'] * df['PCT_AST_3PM']).astype('int8')
    df['UAST_2PM'] = (df['FG2M'] * df['PCT_UAST_2PM']).astype('int8')
    df['UAST_3PM'] = (df['FG3M'] * df['PCT_UAST_3PM']).astype('int8')

    df['POINT_DIFF'] = df['PLUS_MINUS']
    df['RECORD'] = df['WL']
    df['TEAM_SCORE'] = df['PTS']
    
    df = df.drop(columns = ['PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_2PT',
                          'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 'POSS',                       
                          'PCT_AST_2PM', 'PCT_UAST_2PM','PCT_AST_3PM',
                          'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM',
                          'FT_PCT', 'FG_PCT', 'FG3_PCT', 'DREB_PCT',
                          'OREB_PCT', 'REB_PCT', 'AST_PCT', 'AST_TOV', 
                          'AST_RATIO', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
                          'EFG_PCT', 'TS_PCT', 'E_USG_PCT',
                           'MIN', 'PIE', 'CFG_PCT', 'UFG_PCT',
                          'DFG_PCT'])
    
    ## Reorder Columns


    return df


clean_df2 = prep_for_aggregation(clean_df)

clean_df2

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_PACE,contestedShots,contestedShots2pt,contestedShots3pt,deflections,chargesDrawn,screenAssists,screenAssistPoints,looseBallsRecoveredOffensive,looseBallsRecoveredDefensive,looseBallsRecoveredTotal,offensiveBoxOuts,defensiveBoxOuts,boxOutPlayerTeamRebounds,boxOutPlayerRebounds,boxOuts,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,HOME_GAME,FG2M,FG2A,PTS_2PT_MR,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,POINT_DIFF,RECORD,TEAM_SCORE
2632,22016,1610612739,CLE,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,1,45,94,13,35,14,19,11,40,51,31,12,5,15,22,117,29,2016-17,110.0,87.3,22.7,103.58,65.0,46.0,19.0,16.0,0.0,6.0,12.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,17.12,41.0,72.0,108.0,441.0,2.0,0.0,314.0,17.0,39.0,28.0,55.0,10.0,18.0,21.0,16.0,16.0,52.0,1,32,59,12,20,10,12,2,29,1,117
2633,22016,1610612752,NYK,New York Knicks,0021600001,2016-10-25,NYK @ CLE,0,32,87,9,27,15,20,13,29,42,17,6,6,18,22,88,-29,2016-17,87.3,110.0,-22.7,103.58,65.0,39.0,26.0,9.0,2.0,7.0,15.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,17.29,34.0,71.0,104.0,408.0,1.0,1.0,287.0,12.0,31.0,20.0,56.0,21.0,31.0,14.0,16.0,9.0,28.0,0,23,60,18,8,8,14,0,-29,0,88
2634,22016,1610612757,POR,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,1,39,75,13,19,22,22,5,29,34,22,5,3,13,18,113,9,2016-17,121.9,107.2,14.8,94.86,62.0,44.0,18.0,9.0,0.0,18.0,43.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,16.65,19.0,54.0,73.0,362.0,0.0,0.0,255.0,19.0,43.0,20.0,32.0,14.0,20.0,20.0,15.0,4.0,34.0,1,26,56,17,13,8,13,4,9,1,113
2635,22016,1610612762,UTA,Utah Jazz,0021600002,2016-10-25,UTA @ POR,0,40,82,8,24,16,16,6,25,31,19,9,5,14,19,104,-9,2016-17,107.2,121.9,-14.8,94.86,54.0,44.0,10.0,16.0,0.0,10.0,20.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,16.76,19.0,47.0,66.0,425.0,2.0,1.0,310.0,22.0,44.0,18.0,38.0,11.0,18.0,8.0,4.0,8.0,50.0,0,32,58,14,11,8,20,0,-9,0,104
2636,22016,1610612744,GSW,Golden State Warriors,0021600003,2016-10-25,GSW vs. SAS,0,40,85,7,33,13,18,8,27,35,24,11,6,16,19,100,-29,2016-17,99.1,125.9,-26.8,101.68,83.0,62.0,21.0,23.0,0.0,5.0,11.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,16.90,23.0,45.0,67.0,397.0,4.0,1.0,277.0,21.0,38.0,19.0,47.0,20.0,32.0,15.0,4.0,20.0,48.0,1,33,52,18,19,4,13,2,-29,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23043,42023,1610612738,BOS,Boston Celtics,0042300403,2024-06-12,BOS @ DAL,1,38,82,17,46,13,14,6,30,36,26,4,6,9,19,106,7,2023-24,116.3,104.2,12.1,93.10,45.0,29.0,16.0,8.0,0.0,7.0,16.0,1.0,4.0,5.0,0.0,7.0,6.0,3.0,7.0,16.52,12.0,60.0,72.0,405.0,2.0,0.0,296.0,15.0,26.0,23.0,56.0,17.0,28.0,13.0,6.0,12.0,36.0,0,21,36,6,12,13,8,3,7,1,106
23044,42023,1610612742,DAL,Dallas Mavericks,0042300404,2024-06-14,DAL vs. BOS,1,46,91,15,37,15,22,13,39,52,21,7,2,9,17,122,38,2023-24,126.2,87.8,38.4,96.20,37.0,20.0,17.0,15.0,0.0,11.0,28.0,3.0,2.0,5.0,7.0,5.0,11.0,8.0,12.0,17.63,26.0,59.0,85.0,349.0,1.0,2.0,232.0,17.0,35.0,29.0,56.0,11.0,21.0,17.0,16.0,11.0,60.0,1,31,54,1,7,13,23,1,38,1,122
23045,42023,1610612738,BOS,Boston Celtics,0042300404,2024-06-14,BOS @ DAL,0,29,80,14,41,12,13,4,27,31,18,2,5,14,19,84,-38,2023-24,87.8,126.2,-38.4,96.20,44.0,24.0,20.0,11.0,0.0,6.0,17.0,1.0,2.0,3.0,2.0,3.0,5.0,3.0,5.0,17.26,18.0,55.0,71.0,404.0,6.0,2.0,290.0,11.0,27.0,18.0,53.0,20.0,29.0,9.0,2.0,6.0,26.0,0,15,39,4,7,9,7,4,-38,0,84
23046,42023,1610612742,DAL,Dallas Mavericks,0042300405,2024-06-17,DAL @ BOS,0,35,78,11,37,7,13,7,28,35,18,4,4,13,20,88,-18,2023-24,98.1,115.5,-17.4,90.76,43.0,17.0,26.0,13.0,0.0,11.0,26.0,3.0,1.0,4.0,2.0,6.0,8.0,4.0,8.0,16.67,14.0,56.0,70.0,315.0,1.0,0.0,207.0,13.0,22.0,22.0,56.0,16.0,30.

In [246]:
def find_high_correlations(df, threshold=0.7):
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Compute the correlation matrix
    correlation_matrix = numeric_df.corr()
    
    # Create a list to store highly correlated pairs
    high_correlations = []
    
    # Loop through the correlation matrix
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                col1 = correlation_matrix.columns[i]
                col2 = correlation_matrix.columns[j]
                corr_value = correlation_matrix.iloc[i, j]
                high_correlations.append((col1, col2, corr_value))
    
    # Sort the list by absolute correlation value in descending order
    high_correlations.sort(key=lambda x: abs(x[2]), reverse=True)
    
    return high_correlations

# Find highly correlated variables
high_corr_pairs = find_high_correlations(clean_df2, threshold=0.7)

# Print the results
print("Highly correlated variable pairs (correlation > 0.7):")
for col1, col2, corr in high_corr_pairs:
    print(f"{col1} and {col2}: {corr:.2f}")

Highly correlated variable pairs (correlation > 0.7):
REL_E_DEF_RATING_L10 and REL_E_OFF_RATING_opp_L10: 1.00
REL_PLUS_MINUS_L10 and REL_PLUS_MINUS_opp_L10: -1.00
REL_OREB_PCT_opp_L10 and REL_DREB_PCT_L10: -1.00
REL_E_NET_RATING_L10 and REL_E_NET_RATING_opp_L10: -1.00
REL_E_OFF_RATING_L10 and REL_E_DEF_RATING_opp_L10: 1.00
REL_E_PACE_opp_L10 and REL_E_PACE_L10: 1.00
REL_REB_PCT_L10 and REL_REB_PCT_opp_L10: -1.00
REL_OREB_PCT_L10 and REL_DREB_PCT_opp_L10: -1.00
REL_TOV_opp_L10 and REL_TOV_PCT_opp_L10: 0.99
REL_screenAssists_opp_L10 and REL_screenAssistPoints_opp_L10: 0.99
REL_PASS_L10 and REL_TCHS_L10: 0.99
REL_PLUS_MINUS_opp_L10 and REL_E_NET_RATING_opp_L10: 0.99
REL_PLUS_MINUS_L10 and REL_E_NET_RATING_opp_L10: -0.99
REL_E_NET_RATING_L10 and REL_PLUS_MINUS_opp_L10: -0.99
REL_E_NET_RATING_L10 and REL_PLUS_MINUS_L10: 0.99
REL_TOV_PCT_L10 and REL_TOV_L10: 0.98
REL_PASS_opp_L10 and REL_TCHS_opp_L10: 0.98
REL_screenAssists_L10 and REL_screenAssistPoints_L10: 0.98
REL_PTS_L10 and REL_E_DEF_R

In [247]:
high_corr_pairs

[('REL_E_DEF_RATING_L10', 'REL_E_OFF_RATING_opp_L10', 1.0),
 ('REL_PLUS_MINUS_L10', 'REL_PLUS_MINUS_opp_L10', -1.0),
 ('REL_OREB_PCT_opp_L10', 'REL_DREB_PCT_L10', -1.0),
 ('REL_E_NET_RATING_L10', 'REL_E_NET_RATING_opp_L10', -1.0),
 ('REL_E_OFF_RATING_L10', 'REL_E_DEF_RATING_opp_L10', 1.0),
 ('REL_E_PACE_opp_L10', 'REL_E_PACE_L10', 1.0),
 ('REL_REB_PCT_L10', 'REL_REB_PCT_opp_L10', -0.9999999999999998),
 ('REL_OREB_PCT_L10', 'REL_DREB_PCT_opp_L10', -0.9999999999999998),
 ('REL_TOV_opp_L10', 'REL_TOV_PCT_opp_L10', 0.9902291776746244),
 ('REL_screenAssists_opp_L10',
  'REL_screenAssistPoints_opp_L10',
  0.9890193015571056),
 ('REL_PASS_L10', 'REL_TCHS_L10', 0.9884518974321779),
 ('REL_PLUS_MINUS_opp_L10', 'REL_E_NET_RATING_opp_L10', 0.9854770826463026),
 ('REL_PLUS_MINUS_L10', 'REL_E_NET_RATING_opp_L10', -0.9854770826463026),
 ('REL_E_NET_RATING_L10', 'REL_PLUS_MINUS_opp_L10', -0.9854770826463026),
 ('REL_E_NET_RATING_L10', 'REL_PLUS_MINUS_L10', 0.9854770826463026),
 ('REL_TOV_PCT_L10', 'R

In [200]:
def normalize_per_100_poss(df):
    df = df.copy(deep=True)
    
    stats = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM',
       'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', 'contestedShots',
       'contestedShots2pt', 'contestedShots3pt', 'deflections', 'chargesDrawn',
       'screenAssists', 'screenAssistPoints', 'looseBallsRecoveredOffensive',
       'looseBallsRecoveredDefensive', 'looseBallsRecoveredTotal',
       'offensiveBoxOuts', 'defensiveBoxOuts', 'boxOutPlayerTeamRebounds',
       'boxOutPlayerRebounds', 'boxOuts', 'DIST', 'ORBC', 'DRBC', 'RBC',
       'TCHS', 'SAST', 'FTAST', 'PASS', 'CFGM', 'CFGA', 'UFGM', 'UFGA', 'DFGM',
       'DFGA', 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
        'FG2M', 'FG2A', 'PTS_2PT_MR', 'AST_2PM', 'AST_3PM',
       'UAST_2PM', 'UAST_3PM']
    
    df[stats] = 100*df[stats].div(df['E_PACE'], axis=0) 
    
    return df
 
normalized_df = normalize_per_100_poss(clean_df2)
normalized_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_PACE,contestedShots,contestedShots2pt,contestedShots3pt,deflections,chargesDrawn,screenAssists,screenAssistPoints,looseBallsRecoveredOffensive,looseBallsRecoveredDefensive,looseBallsRecoveredTotal,offensiveBoxOuts,defensiveBoxOuts,boxOutPlayerTeamRebounds,boxOutPlayerRebounds,boxOuts,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,HOME_GAME,FG2M,FG2A,PTS_2PT_MR,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,POINT_DIFF,RECORD,TEAM_SCORE
2632,22016,1610612739,CLE,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,1,43.444680,90.751110,12.550685,33.790307,13.516123,18.343310,10.619811,38.617494,49.237304,29.928558,11.585248,4.827187,14.481560,21.239622,112.956169,27.997683,2016-17,110.0,87.3,22.7,103.58,62.753427,44.410118,18.343310,15.446997,0.000000,5.792624,11.585248,0.000000,0.000000,11.585248,0.000000,0.000000,0.000000,0.000000,0.000000,16.528287,39.582931,69.511489,104.267233,425.757868,1.930875,0.000000,303.147326,16.412435,37.652056,27.032246,53.099054,9.654373,17.377872,20.274184,15.446997,15.446997,50.202742,1,30.893995,56.960803,11.585248,19.308747,9.654373,11.585248,1.930875,29,1,117
2633,22016,1610612752,NYK,New York Knicks,0021600001,2016-10-25,NYK @ CLE,0,30.893995,83.993049,8.688936,26.066808,14.481560,19.308747,12.550685,27.997683,40.548368,16.412435,5.792624,5.792624,17.377872,21.239622,84.958486,-27.997683,2016-17,87.3,110.0,-22.7,103.58,62.753427,37.652056,25.101371,8.688936,1.930875,6.758061,14.481560,0.000000,0.000000,9.654373,0.000000,0.000000,0.000000,0.000000,0.000000,16.692412,32.824870,68.546051,100.405484,393.898436,0.965437,0.965437,277.080517,11.585248,29.928558,19.308747,54.064491,20.274184,29.928558,13.516123,15.446997,8.688936,27.032246,0,22.205059,57.926241,17.377872,7.723499,7.723499,13.516123,0.000000,-29,0,88
2634,22016,1610612757,POR,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,1,41.113219,79.063884,13.704406,20.029517,23.192073,23.192073,5.270926,30.571368,35.842294,23.192073,5.270926,3.162555,13.704406,18.975332,119.122918,9.487666,2016-17,121.9,107.2,14.8,94.86,65.359477,46.384145,18.975332,9.487666,0.000000,18.975332,45.329960,0.000000,0.000000,4.216740,0.000000,0.000000,0.000000,0.000000,0.000000,17.552182,20.029517,56.925996,76.955513,381.615012,0.000000,0.000000,268.817204,20.029517,45.329960,21.083702,33.733924,14.758592,21.083702,21.083702,15.812777,4.216740,35.842294,1,27.408813,59.034366,17.921147,13.704406,8.433481,13.704406,4.216740,9,1,113
2635,22016,1610612762,UTA,Utah Jazz,0021600002,2016-10-25,UTA @ POR,0,42.167405,86.443179,8.433481,25.300443,16.866962,16.866962,6.325111,26.354628,32.679739,20.029517,9.487666,5.270926,14.758592,20.029517,109.635252,-9.487666,2016-17,107.2,121.9,-14.8,94.86,56.925996,46.384145,10.541851,16.866962,0.000000,10.541851,21.083702,0.000000,0.000000,7.379296,0.000000,0.000000,0.000000,0.000000,0.000000,17.668143,20.029517,49.546700,69.576218,448.028674,2.108370,1.054185,326.797386,23.192073,46.384145,18.975332,40.059034,11.596036,18.975332,8.433481,4.216740,8.433481,52.709256,0,33.733924,61.142737,14.758592,11.596036,8.433481,21.083702,0.000000,-9,0,104
2636,22016,1610612744,GSW,Golden State Warriors,0021600003,2016-10-25,GSW vs. SAS,0,39.339103,83.595594,6.884343,32.454760,12.785208,17.702596,7.867821,26.553895,34.421715,23.603462,10.818253,5.900865,15.735641,18.686074,98.347758,-28.520850,2016-17,99.1,125.9,-26.8,101.68,81.628639,60.975610,20.653029,22.619984,0.000000,4.917388,10.818253,0.000000,0.000000,7.867821,0.000000,0.000000,0.000000,0.000000,0.000000,16.620771,22.619984,44.256491,65.892998,390.440598,3.933910,0.983478,272.423289,20.653029,37.372148,18.686074,46.223446,19.669552,31.471282,14.752164,3.933910,19.669552,47.206924,1,32.454760,51.140834,17.702596,18.686074,3.933910,12.

In [98]:
normalized_df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM',
       'FTA', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', 'SEASON', 'E_PACE', 'contestedShots', 'contestedShots2pt',
       'contestedShots3pt', 'deflections', 'chargesDrawn', 'screenAssists',
       'screenAssistPoints', 'looseBallsRecoveredOffensive',
       'looseBallsRecoveredDefensive', 'looseBallsRecoveredTotal',
       'offensiveBoxOuts', 'defensiveBoxOuts', 'boxOutPlayerTeamRebounds',
       'boxOutPlayerRebounds', 'boxOuts', 'DIST', 'ORBC', 'DRBC', 'RBC',
       'TCHS', 'SAST', 'FTAST', 'PASS', 'CFGM', 'CFGA', 'UFGM', 'UFGA', 'DFGM',
       'DFGA', 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'HOME_GAME', 'FG2M', 'FG2A', 'PTS_2PT_MR', 'AST_2PM', 'AST_3PM',
       'UAST_2PM', 'UAST_3PM', 'POINT_DIFF', 'RECORD', 'TEAM_SCORE'],
      dtype='object')

## Add Betting Data

In [201]:
def load_betting_data(conn):
    betting_data = pd.read_sql("SELECT * FROM nba_odds", conn)
    betting_data = betting_data[['season', 'date', 'status', 'home_team_abbr', 'away_team_abbr', 'home_spread_draftkings',
                                'home_spread_odds_draftkings',
                                'away_spread_draftkings',
                                'away_spread_odds_draftkings',
                                'home_ml_draftkings',
                                'away_ml_draftkings',
                                'total_draftkings',
                                'over_odds_draftkings',
                                'under_odds_draftkings']]
    return betting_data


connection = sqlite3.connect("nba_stats.db")
betting_data = load_betting_data(connection)
betting_data

connection.close()

In [202]:
def convert_american_to_decimal(x):
    return np.where(x>0, (100+x)/100, 1+(100.0/-x))          

In [203]:
normalized_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_PACE,contestedShots,contestedShots2pt,contestedShots3pt,deflections,chargesDrawn,screenAssists,screenAssistPoints,looseBallsRecoveredOffensive,looseBallsRecoveredDefensive,looseBallsRecoveredTotal,offensiveBoxOuts,defensiveBoxOuts,boxOutPlayerTeamRebounds,boxOutPlayerRebounds,boxOuts,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,HOME_GAME,FG2M,FG2A,PTS_2PT_MR,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,POINT_DIFF,RECORD,TEAM_SCORE
2632,22016,1610612739,CLE,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,1,43.444680,90.751110,12.550685,33.790307,13.516123,18.343310,10.619811,38.617494,49.237304,29.928558,11.585248,4.827187,14.481560,21.239622,112.956169,27.997683,2016-17,110.0,87.3,22.7,103.58,62.753427,44.410118,18.343310,15.446997,0.000000,5.792624,11.585248,0.000000,0.000000,11.585248,0.000000,0.000000,0.000000,0.000000,0.000000,16.528287,39.582931,69.511489,104.267233,425.757868,1.930875,0.000000,303.147326,16.412435,37.652056,27.032246,53.099054,9.654373,17.377872,20.274184,15.446997,15.446997,50.202742,1,30.893995,56.960803,11.585248,19.308747,9.654373,11.585248,1.930875,29,1,117
2633,22016,1610612752,NYK,New York Knicks,0021600001,2016-10-25,NYK @ CLE,0,30.893995,83.993049,8.688936,26.066808,14.481560,19.308747,12.550685,27.997683,40.548368,16.412435,5.792624,5.792624,17.377872,21.239622,84.958486,-27.997683,2016-17,87.3,110.0,-22.7,103.58,62.753427,37.652056,25.101371,8.688936,1.930875,6.758061,14.481560,0.000000,0.000000,9.654373,0.000000,0.000000,0.000000,0.000000,0.000000,16.692412,32.824870,68.546051,100.405484,393.898436,0.965437,0.965437,277.080517,11.585248,29.928558,19.308747,54.064491,20.274184,29.928558,13.516123,15.446997,8.688936,27.032246,0,22.205059,57.926241,17.377872,7.723499,7.723499,13.516123,0.000000,-29,0,88
2634,22016,1610612757,POR,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,1,41.113219,79.063884,13.704406,20.029517,23.192073,23.192073,5.270926,30.571368,35.842294,23.192073,5.270926,3.162555,13.704406,18.975332,119.122918,9.487666,2016-17,121.9,107.2,14.8,94.86,65.359477,46.384145,18.975332,9.487666,0.000000,18.975332,45.329960,0.000000,0.000000,4.216740,0.000000,0.000000,0.000000,0.000000,0.000000,17.552182,20.029517,56.925996,76.955513,381.615012,0.000000,0.000000,268.817204,20.029517,45.329960,21.083702,33.733924,14.758592,21.083702,21.083702,15.812777,4.216740,35.842294,1,27.408813,59.034366,17.921147,13.704406,8.433481,13.704406,4.216740,9,1,113
2635,22016,1610612762,UTA,Utah Jazz,0021600002,2016-10-25,UTA @ POR,0,42.167405,86.443179,8.433481,25.300443,16.866962,16.866962,6.325111,26.354628,32.679739,20.029517,9.487666,5.270926,14.758592,20.029517,109.635252,-9.487666,2016-17,107.2,121.9,-14.8,94.86,56.925996,46.384145,10.541851,16.866962,0.000000,10.541851,21.083702,0.000000,0.000000,7.379296,0.000000,0.000000,0.000000,0.000000,0.000000,17.668143,20.029517,49.546700,69.576218,448.028674,2.108370,1.054185,326.797386,23.192073,46.384145,18.975332,40.059034,11.596036,18.975332,8.433481,4.216740,8.433481,52.709256,0,33.733924,61.142737,14.758592,11.596036,8.433481,21.083702,0.000000,-9,0,104
2636,22016,1610612744,GSW,Golden State Warriors,0021600003,2016-10-25,GSW vs. SAS,0,39.339103,83.595594,6.884343,32.454760,12.785208,17.702596,7.867821,26.553895,34.421715,23.603462,10.818253,5.900865,15.735641,18.686074,98.347758,-28.520850,2016-17,99.1,125.9,-26.8,101.68,81.628639,60.975610,20.653029,22.619984,0.000000,4.917388,10.818253,0.000000,0.000000,7.867821,0.000000,0.000000,0.000000,0.000000,0.000000,16.620771,22.619984,44.256491,65.892998,390.440598,3.933910,0.983478,272.423289,20.653029,37.372148,18.686074,46.223446,19.669552,31.471282,14.752164,3.933910,19.669552,47.206924,1,32.454760,51.140834,17.702596,18.686074,3.933910,12.

In [204]:
def clean_betting_data(df):
    abbr_mapping = {'BK':'BKN',
                    'GS':'GSW',
                    'NO':'NOP',
                    'NY':'NYK',
                    'PHO':'PHX',
                    'SA':'SAS'
                    }

    df['home_team_abbr'] = df['home_team_abbr'].replace(abbr_mapping)
    df['away_team_abbr'] = df['away_team_abbr'].replace(abbr_mapping)

    df['date'] = pd.to_datetime(df['date']).dt.date
    df['GAME_DATE'] = df['date'] - pd.Timedelta(days=1)

    df['home_spread_odds_draftkings'] = convert_american_to_decimal(df['home_spread_odds_draftkings'])
    df['away_spread_odds_draftkings'] = convert_american_to_decimal(df['away_spread_odds_draftkings'])

    df['home_ml_draftkings'] = convert_american_to_decimal(df['home_ml_draftkings'])
    df['away_ml_draftkings'] = convert_american_to_decimal(df['away_ml_draftkings'])
    
    df['over_odds_draftkings'] = convert_american_to_decimal(df['over_odds_draftkings'])
    df['under_odds_draftkings'] = convert_american_to_decimal(df['under_odds_draftkings'])

    return df

In [205]:
clean_betting_df = clean_betting_data(betting_data)

In [206]:
clean_betting_df.head()

,season,date,status,home_team_abbr,away_team_abbr,home_spread_draftkings,home_spread_odds_draftkings,away_spread_draftkings,away_spread_odds_draftkings,home_ml_draftkings,away_ml_draftkings,total_draftkings,over_odds_draftkings,under_odds_draftkings,GAME_DATE
0,2019-20,2020-10-11,Final,MIA,LAL,6.0,1.909091,-6.0,1.909091,3.150000,1.377358,214.5,1.925926,1.884956,2020-10-10
1,2019-20,2020-10-11,Final,MIA,LAL,6.0,1.909091,-6.0,1.909091,3.150000,1.377358,214.5,1.925926,1.884956,2020-10-10
2,2019-20,2020-10-10,Final,LAL,MIA,-7.0,1.892857,7.0,1.925926,1.298507,3.700000,215.5,1.892857,1.917431,2020-10-09
3,2019-20,2020-10-07,Final,MIA,LAL,7.5,1.909091,-7.5,1.909091,3.750000,1.298507,218.5,1.884956,1.925926,2020-10-06
4,2019-20,2020-10-04,Final,MIA,LAL,9.5,1.909091,-9.5,1.909091,4.500000,1.219780,219.5,1.884956,1.925926,2020-10-03


In [207]:
normalized_df['GAME_DATE'] = pd.to_datetime(normalized_df['GAME_DATE'])

betting_data['season'].unique()

array(['2019-20', '2020-21', '2021-22', '2022-23', '2023-24'],
      dtype=object)

In [208]:
clean_betting_df[['season', 'date', 'GAME_DATE', 'home_team_abbr', 'away_team_abbr']]

,season,date,GAME_DATE,home_team_abbr,away_team_abbr
0,2019-20,2020-10-11,2020-10-10,MIA,LAL
1,2019-20,2020-10-11,2020-10-10,MIA,LAL
2,2019-20,2020-10-10,2020-10-09,LAL,MIA
3,2019-20,2020-10-07,2020-10-06,MIA,LAL
4,2019-20,2020-10-04,2020-10-03,MIA,LAL
...,...,...,...,...,...
6456,2023-24,2024-06-07,2024-06-06,BOS,DAL
6457,2023-24,2024-06-10,2024-06-09,BOS,DAL
6458,2023-24,2024-06-13,2024-06-12,DAL,BOS
6459,2023-24,2024-06-15,2024-06-14,DAL,BOS


In [209]:
# Create copy of input DataFrames to avoid modifying originals
clean_boxscores = normalized_df.copy()
clean_betting_df = clean_betting_df.copy()

# Ensure date columns are in the same format
clean_boxscores['GAME_DATE'] = pd.to_datetime(clean_boxscores['GAME_DATE']) #.dt.tz_localize(None)
clean_betting_df['GAME_DATE'] = pd.to_datetime(clean_betting_df['GAME_DATE']) #.dt.tz_localize(None)    

clean_boxscores['HOME_TEAM'] = clean_boxscores['MATCHUP'].apply(
    lambda x: x[:3] if 'vs' in x else x[-3:])
clean_boxscores['AWAY_TEAM'] = clean_boxscores['MATCHUP'].apply(
    lambda x: x[:3] if '@' in x else x[-3:])


merged_df = pd.merge(clean_boxscores, clean_betting_df, how='left', 
                        left_on=['HOME_TEAM', 'AWAY_TEAM', 'GAME_DATE'],
                        right_on=['home_team_abbr', 'away_team_abbr', 'GAME_DATE'])


merged_df['ML'] = merged_df.apply(lambda row: row['home_ml_draftkings'] if row['HOME_GAME'] == 1
                                    else row['away_ml_draftkings'], axis=1)

merged_df['SPREAD'] = merged_df.apply(lambda row: row['home_spread_odds_draftkings'] if row['HOME_GAME'] == 1
                                        else -row['away_spread_odds_draftkings'], axis=1)


merged_df['SPREAD_ODDS'] = merged_df.apply(lambda row: row['home_spread_draftkings'] if row['HOME_GAME'] == 1
                                        else -row['away_spread_draftkings'], axis=1)


merged_df = merged_df.drop(columns=['HOME_TEAM', 'AWAY_TEAM', 'date'])

merged_df['ATS_DIFF'] = merged_df['POINT_DIFF'] + merged_df['SPREAD']

merged_df['TEAM_COVERED'] = (merged_df['ATS_DIFF'] > 0).astype(int)


In [210]:
def merge_betting_and_boxscore_data(clean_betting_df, clean_boxscores):
    
    # Create copy of input DataFrames to avoid modifying originals
    clean_boxscores = clean_boxscores.copy()
    clean_betting_df = clean_betting_df.copy()

    # Ensure date columns are in the same format
    clean_boxscores['GAME_DATE'] = pd.to_datetime(clean_boxscores['GAME_DATE']) #.dt.tz_localize(None)
    clean_betting_df['GAME_DATE'] = pd.to_datetime(clean_betting_df['GAME_DATE']) #.dt.tz_localize(None)    

    clean_boxscores['HOME_TEAM'] = clean_boxscores['MATCHUP'].apply(
        lambda x: x[:3] if 'vs' in x else x[-3:])
    clean_boxscores['AWAY_TEAM'] = clean_boxscores['MATCHUP'].apply(
        lambda x: x[:3] if '@' in x else x[-3:])


    merged_df = pd.merge(clean_boxscores, clean_betting_df, how='left', 
                            left_on=['HOME_TEAM', 'AWAY_TEAM', 'GAME_DATE'],
                            right_on=['home_team_abbr', 'away_team_abbr', 'GAME_DATE'])

    merged_df['ML'] = merged_df.apply(lambda row: row['home_ml_draftkings'] if row['HOME_GAME'] == 1
                                      else row['away_ml_draftkings'], axis=1)

    merged_df['SPREAD'] = merged_df.apply(lambda row: row['home_spread_odds_draftkings'] if row['HOME_GAME'] == 1
                                          else -row['away_spread_odds_draftkings'], axis=1)
    
    
    merged_df['SPREAD_ODDS'] = merged_df.apply(lambda row: row['home_spread_draftkings'] if row['HOME_GAME'] == 1
                                          else -row['away_spread_draftkings'], axis=1)
    

    merged_df = merged_df.drop(columns=['HOME_TEAM', 'AWAY_TEAM', 'date'])

    merged_df['ATS_DIFF'] = merged_df['POINT_DIFF'] + merged_df['SPREAD']

    merged_df['TEAM_COVERED'] = (merged_df['ATS_DIFF'] > 0).astype(int)
    
#     merged_df.loc[merged_df['ATS_DIFF'] == 0.0, 'TEAM_COVERED'] = 0.5

    return merged_df


merged_df = merge_betting_and_boxscore_data(clean_betting_df, clean_boxscores=normalized_df)

merged_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_PACE,contestedShots,contestedShots2pt,contestedShots3pt,deflections,chargesDrawn,screenAssists,screenAssistPoints,looseBallsRecoveredOffensive,looseBallsRecoveredDefensive,looseBallsRecoveredTotal,offensiveBoxOuts,defensiveBoxOuts,boxOutPlayerTeamRebounds,boxOutPlayerRebounds,boxOuts,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,HOME_GAME,FG2M,FG2A,PTS_2PT_MR,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,POINT_DIFF,RECORD,TEAM_SCORE,season,status,home_team_abbr,away_team_abbr,home_spread_draftkings,home_spread_odds_draftkings,away_spread_draftkings,away_spread_odds_draftkings,home_ml_draftkings,away_ml_draftkings,total_draftkings,over_odds_draftkings,under_odds_draftkings,ML,SPREAD,SPREAD_ODDS,ATS_DIFF,TEAM_COVERED
0,22016,1610612739,CLE,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,1,43.444680,90.751110,12.550685,33.790307,13.516123,18.343310,10.619811,38.617494,49.237304,29.928558,11.585248,4.827187,14.481560,21.239622,112.956169,27.997683,2016-17,110.0,87.3,22.7,103.58,62.753427,44.410118,18.343310,15.446997,0.000000,5.792624,11.585248,0.000000,0.000000,11.585248,0.000000,0.000000,0.000000,0.000000,0.000000,16.528287,39.582931,69.511489,104.267233,425.757868,1.930875,0.000000,303.147326,16.412435,37.652056,27.032246,53.099054,9.654373,17.377872,20.274184,15.446997,15.446997,50.202742,1,30.893995,56.960803,11.585248,19.308747,9.654373,11.585248,1.930875,29,1,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,22016,1610612752,NYK,New York Knicks,0021600001,2016-10-25,NYK @ CLE,0,30.893995,83.993049,8.688936,26.066808,14.481560,19.308747,12.550685,27.997683,40.548368,16.412435,5.792624,5.792624,17.377872,21.239622,84.958486,-27.997683,2016-17,87.3,110.0,-22.7,103.58,62.753427,37.652056,25.101371,8.688936,1.930875,6.758061,14.481560,0.000000,0.000000,9.654373,0.000000,0.000000,0.000000,0.000000,0.000000,16.692412,32.824870,68.546051,100.405484,393.898436,0.965437,0.965437,277.080517,11.585248,29.928558,19.308747,54.064491,20.274184,29.928558,13.516123,15.446997,8.688936,27.032246,0,22.205059,57.926241,17.377872,7.723499,7.723499,13.516123,0.000000,-29,0,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,22016,1610612757,POR,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,1,41.113219,79.063884,13.704406,20.029517,23.192073,23.192073,5.270926,30.571368,35.842294,23.192073,5.270926,3.162555,13.704406,18.975332,119.122918,9.487666,2016-17,121.9,107.2,14.8,94.86,65.359477,46.384145,18.975332,9.487666,0.000000,18.975332,45.329960,0.000000,0.000000,4.216740,0.000000,0.000000,0.000000,0.000000,0.000000,17.552182,20.029517,56.925996,76.955513,381.615012,0.000000,0.000000,268.817204,20.029517,45.329960,21.083702,33.733924,14.758592,21.083702,21.083702,15.812777,4.216740,35.842294,1,27.408813,59.034366,17.921147,13.704406,8.433481,13.704406,4.216740,9,1,113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,22016,1610612762,UTA,Utah Jazz,0021600002,2016-10-25,UTA @ POR,0,42.167405,86.443179,8.433481,25.300443,16.866962,16.866962,6.325111,26.354628,32.679739,20.029517,9.487666,5.270926,14.758592,20.029517,109.635252,-9.487666,2016-17,107.2,121.9,-14.8,94.86,56.925996,46.384145,10.541851,16.866962,0.000000,10.541851,21.083702,0.000000,0.000000,7.379296,0.000000,0.000000,0.000000,0.000000,0.000000,17.668143,20.029517,49.546700,69.576218,448.028674,2.108370,1.054185,326.797386,23.192073,46.384145,18.975332,40.059034,11.596036,18.975332,8.433481,4.216740,8.433481,52.709256,0,33.733924,61.142737,14.758592,11.596036,8.433481,21.083702,0.000000,-9,0,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,22016,1610612744,GSW,Golden State Warriors,0021600003,2016-10-25,GSW vs. SAS,0,39.339103,

In [211]:
def create_matchups(df):
    """This function makes each row a matchup between 
    team and opp"""
    df = df.copy()
    
    
    matchups = pd.merge(df, df, on=['GAME_ID'], suffixes=['', '_opp'])
    matchups = matchups.loc[matchups['TEAM_ABBREVIATION'] != matchups['TEAM_ABBREVIATION_opp']]

    matchups = matchups.drop(columns = ['SEASON_opp', 'SEASON_ID_opp',
                             'TEAM_ABBREVIATION_opp', 'GAME_DATE_opp',
                             'MATCHUP_opp', 'HOME_GAME_opp', 'TEAM_NAME_opp',
                                       'TEAM_ID_opp', 'WL_opp']
                 )
    
    matchups
    
    return matchups


matchups = create_matchups(normalized_df)
matchups.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,SEASON,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_PACE,contestedShots,contestedShots2pt,contestedShots3pt,deflections,chargesDrawn,screenAssists,screenAssistPoints,looseBallsRecoveredOffensive,looseBallsRecoveredDefensive,looseBallsRecoveredTotal,offensiveBoxOuts,defensiveBoxOuts,boxOutPlayerTeamRebounds,boxOutPlayerRebounds,boxOuts,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,HOME_GAME,FG2M,FG2A,PTS_2PT_MR,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,POINT_DIFF,RECORD,TEAM_SCORE,FGM_opp,FGA_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,REB_opp,AST_opp,STL_opp,BLK_opp,TOV_opp,PF_opp,PTS_opp,PLUS_MINUS_opp,E_OFF_RATING_opp,E_DEF_RATING_opp,E_NET_RATING_opp,E_PACE_opp,contestedShots_opp,contestedShots2pt_opp,contestedShots3pt_opp,deflections_opp,chargesDrawn_opp,screenAssists_opp,screenAssistPoints_opp,looseBallsRecoveredOffensive_opp,looseBallsRecoveredDefensive_opp,looseBallsRecoveredTotal_opp,offensiveBoxOuts_opp,defensiveBoxOuts_opp,boxOutPlayerTeamRebounds_opp,boxOutPlayerRebounds_opp,boxOuts_opp,DIST_opp,ORBC_opp,DRBC_opp,RBC_opp,TCHS_opp,SAST_opp,FTAST_opp,PASS_opp,CFGM_opp,CFGA_opp,UFGM_opp,UFGA_opp,DFGM_opp,DFGA_opp,PTS_OFF_TOV_opp,PTS_2ND_CHANCE_opp,PTS_FB_opp,PTS_PAINT_opp,FG2M_opp,FG2A_opp,PTS_2PT_MR_opp,AST_2PM_opp,AST_3PM_opp,UAST_2PM_opp,UAST_3PM_opp,POINT_DIFF_opp,RECORD_opp,TEAM_SCORE_opp
1,22016,1610612739,CLE,Cleveland Cavaliers,0021600001,2016-10-25,CLE vs. NYK,1,43.444680,90.751110,12.550685,33.790307,13.516123,18.343310,10.619811,38.617494,49.237304,29.928558,11.585248,4.827187,14.481560,21.239622,112.956169,27.997683,2016-17,110.0,87.3,22.7,103.58,62.753427,44.410118,18.343310,15.446997,0.000000,5.792624,11.585248,0.0,0.0,11.585248,0.0,0.0,0.0,0.0,0.0,16.528287,39.582931,69.511489,104.267233,425.757868,1.930875,0.000000,303.147326,16.412435,37.652056,27.032246,53.099054,9.654373,17.377872,20.274184,15.446997,15.446997,50.202742,1,30.893995,56.960803,11.585248,19.308747,9.654373,11.585248,1.930875,29,1,117,30.893995,83.993049,8.688936,26.066808,14.481560,19.308747,12.550685,27.997683,40.548368,16.412435,5.792624,5.792624,17.377872,21.239622,84.958486,-27.997683,87.3,110.0,-22.7,103.58,62.753427,37.652056,25.101371,8.688936,1.930875,6.758061,14.481560,0.0,0.0,9.654373,0.0,0.0,0.0,0.0,0.0,16.692412,32.824870,68.546051,100.405484,393.898436,0.965437,0.965437,277.080517,11.585248,29.928558,19.308747,54.064491,20.274184,29.928558,13.516123,15.446997,8.688936,27.032246,22.205059,57.926241,17.377872,7.723499,7.723499,13.516123,0.000000,-29,0,88
2,22016,1610612752,NYK,New York Knicks,0021600001,2016-10-25,NYK @ CLE,0,30.893995,83.993049,8.688936,26.066808,14.481560,19.308747,12.550685,27.997683,40.548368,16.412435,5.792624,5.792624,17.377872,21.239622,84.958486,-27.997683,2016-17,87.3,110.0,-22.7,103.58,62.753427,37.652056,25.101371,8.688936,1.930875,6.758061,14.481560,0.0,0.0,9.654373,0.0,0.0,0.0,0.0,0.0,16.692412,32.824870,68.546051,100.405484,393.898436,0.965437,0.965437,277.080517,11.585248,29.928558,19.308747,54.064491,20.274184,29.928558,13.516123,15.446997,8.688936,27.032246,0,22.205059,57.926241,17.377872,7.723499,7.723499,13.516123,0.000000,-29,0,88,43.444680,90.751110,12.550685,33.790307,13.516123,18.343310,10.619811,38.617494,49.237304,29.928558,11.585248,4.827187,14.481560,21.239622,112.956169,27.997683,110.0,87.3,22.7,103.58,62.753427,44.410118,18.343310,15.446997,0.000000,5.792624,11.585248,0.0,0.0,11.585248,0.0,0.0,0.0,0.0,0.0,16.528287,39.582931,69.511489,104.267233,425.757868,1.930875,0.000000,303.147326,16.412435,37.652056,27.032246,53.099054,9.654373,17.377872,20.274184,15.446997,15.446997,50.202742,30.893995,56.960803,11.585248,19.308747,9.654373,11.585248,1.930875,29,1,117
5,22016,1610612757,POR,Portland Trail Blazers,0021600002,2016-10-25,POR vs. UTA,1,41.113219,79.063884,13.704406

In [212]:
def build_team_avg_stats_df(df: pd.DataFrame, span: int = 10) -> pd.DataFrame:
    """
    Calculate the average statistics for each team up to (but not including) the given date.
    
    Args:
    df (pd.DataFrame): Input DataFrame containing team statistics.
    span (int): Number of games to consider for the exponential weighted moving average. Default is 10.
    
    Returns:
    pd.DataFrame: DataFrame with average team statistics.
    """
    df = df.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE']).reset_index(drop=True)

    # Define columns to drop and keep
    drop_cols = ['SEASON_ID', 'SEASON', 'TEAM_ID', 'TEAM_NAME', 'GAME_ID', 'MATCHUP', 
                 'HOME_GAME', 'TEAM_SCORE', 'GAME_DATE', 'POINT_DIFF', 'WL', 
                 'TEAM_SCORE_opp', 'POINT_DIFF_opp', 'RECORD', 'RECORD_opp']
    
    keep_cols = ['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE', 'GAME_ID', 'MATCHUP', 
                 'HOME_GAME', 'TEAM_SCORE', 'RECORD', 'POINT_DIFF', 'WL']

    # Select stats columns
    stats = df.drop(columns=drop_cols)
    stats_cols = stats.columns[1:]  # Exclude 'TEAM_ABBREVIATION'

    # Calculate exponential weighted moving average for each stat
    avg_stats = stats.groupby('TEAM_ABBREVIATION')[stats_cols].transform(
        lambda x: x.ewm(span=span).mean()
    )

    # Add matchup info and calculate win percentage
    result = df[keep_cols].copy()
    result = pd.concat([result, avg_stats], axis=1)
    result[f'WIN_PCT_L{span}'] = result.groupby('TEAM_ABBREVIATION')['RECORD'].transform(
        lambda x: x.rolling(window=span).mean()
    )
    result = result.drop(columns='RECORD')

    # Shift stats to avoid look-ahead bias
    shift_cols = [col for col in result.columns if col not in keep_cols + [f'WIN_PCT_L{span}']]
    result[shift_cols] = result.groupby('TEAM_ABBREVIATION')[shift_cols].shift(1)

    # Rename columns
    result = result.add_suffix(f'_L{span}')
    rename_dict = {f'{col}_L{span}': col for col in keep_cols}
    result = result.rename(columns=rename_dict)

    return result

# Usage

start = time.time()
team_stats_ewa = build_team_avg_stats_df(matchups, span=10)
end = time.time()
print(f"Time taken: {end - start} seconds")

Time taken: 0.5773587226867676 seconds


In [266]:
def add_percentage_features(df, span):
    """Add the following features for both team and opp:
    OREB_PCT, DREB_PCT, REB_PCT, TS_PCT, EFG_PCT, AST_RATIO, TOV_PCT, PIE.
    """
    
    df = df.copy()
    
    df[f'FG2_PCT_L{span}'] = df[f'FG2M_L{span}'] / df[f'FG2A_L{span}']
    df[f'FG3_PCT_L{span}'] = df[f'FG3M_L{span}'] / df[f'FG3A_L{span}']
    
    df['OREB_PCT_L{}'.format(span)] = df['OREB_L{}'.format(span)] / (df['OREB_L{}'.format(span)] + df['DREB_opp_L{}'.format(span)])
    df['OREB_PCT_opp_L{}'.format(span)] = df['OREB_opp_L{}'.format(span)] / (df['OREB_opp_L{}'.format(span)] + df['DREB_L{}'.format(span)])

    df['DREB_PCT_L{}'.format(span)] = df['DREB_L{}'.format(span)] / (df['DREB_L{}'.format(span)] + df['OREB_opp_L{}'.format(span)])
    df['DREB_PCT_opp_L{}'.format(span)] = df['DREB_opp_L{}'.format(span)] / (df['DREB_opp_L{}'.format(span)] + df['OREB_L{}'.format(span)])

    df['REB_PCT_L{}'.format(span)] = df['REB_L{}'.format(span)] / (df['REB_L{}'.format(span)] + df['REB_opp_L{}'.format(span)])
    df['REB_PCT_opp_L{}'.format(span)] = df['REB_opp_L{}'.format(span)] / (df['REB_opp_L{}'.format(span)] + df['REB_L{}'.format(span)])

    df['TS_PCT_L{}'.format(span)] = df['PTS_L{}'.format(span)] / ((2*(df['FG2A_L{}'.format(span)] + df['FG3A_L{}'.format(span)]) + 0.44*df['FTA_L{}'.format(span)]))
    
    df['TS_PCT_opp_L{}'.format(span)] = df['PTS_opp_L{}'.format(span)] / ((2*(df['FG2A_opp_L{}'.format(span)] + df['FG3A_opp_L{}'.format(span)]) + 0.44*df['FTA_opp_L{}'.format(span)]))

    df['EFG_PCT_L{}'.format(span)] = (df['FG2M_L{}'.format(span)] + 1.5*df['FG3M_L{}'.format(span)]) / (df['FG2A_L{}'.format(span)]
                                                                    + df['FG3A_L{}'.format(span)])
    df['EFG_PCT_opp_L{}'.format(span)] = (df['FG2M_opp_L{}'.format(span)] + 1.5*df['FG3M_opp_L{}'.format(span)]) / (df['FG2A_opp_L{}'.format(span)] 
                                                                 + df['FG3A_opp_L{}'.format(span)])

    df['AST_RATIO_L{}'.format(span)] = (df['AST_L{}'.format(span)] * 100) / df['E_PACE_L{}'.format(span)]
    df['AST_RATIO_opp_L{}'.format(span)] = (df['AST_opp_L{}'.format(span)] * 100) / df['E_PACE_opp_L{}'.format(span)]

    df['TOV_PCT_L{}'.format(span)] = 100*df['TOV_L{}'.format(span)] / (df['FG2A_L{}'.format(span)] 
                                               + df['FG3A_L{}'.format(span)] 
                                               + 0.44*df['FTA_L{}'.format(span)] 
                                               + df['TOV_L{}'.format(span)])
    
    df['TOV_PCT_opp_L{}'.format(span)] = 100*df['TOV_opp_L{}'.format(span)] / (df['FG2A_opp_L{}'.format(span)] 
                                             + df['FG3A_opp_L{}'.format(span)] 
                                             + 0.44*df['FTA_opp_L{}'.format(span)] 
                                             + df['TOV_opp_L{}'.format(span)])
    
    
    df['PIE_L{}'.format(span)] = ((df['PTS_L{}'.format(span)] + df['FG2M_L{}'.format(span)] + df['FG3M_L{}'.format(span)] + df['FTM_L{}'.format(span)] 
                 - df['FG2A_L{}'.format(span)] - df['FG3A_L{}'.format(span)] - df['FTA_L{}'.format(span)] 
                 + df['DREB_L{}'.format(span)] + df['OREB_L{}'.format(span)]/2
                + df['AST_L{}'.format(span)] + df['STL_L{}'.format(span)] + df['BLK_L{}'.format(span)]/2
                - df['PF_L{}'.format(span)] - df['TOV_L{}'.format(span)]) 
                 / (df['PTS_L{}'.format(span)] + df['PTS_opp_L{}'.format(span)] + df['FG2M_L{}'.format(span)] + df['FG2M_opp_L{}'.format(span)]
                   + df['FG3M_L{}'.format(span)] + df['FG3M_opp_L{}'.format(span)] + df['FTM_L{}'.format(span)] + df['FTM_opp_L{}'.format(span)]
                   - df['FG2A_L{}'.format(span)] - df['FG2A_opp_L{}'.format(span)] - df['FG3A_L{}'.format(span)] - df['FG3A_opp_L{}'.format(span)] 
                    - df['FTA_L{}'.format(span)] - df['FTA_opp_L{}'.format(span)] + df['DREB_L{}'.format(span)] + df['DREB_opp_L{}'.format(span)]
                    + (df['OREB_L{}'.format(span)]+df['OREB_opp_L{}'.format(span)])/2 + df['AST_L{}'.format(span)] + df['AST_opp_L{}'.format(span)]
                    + df['STL_L{}'.format(span)] + df['STL_opp_L{}'.format(span)] + (df['BLK_L{}'.format(span)] + df['BLK_opp_L{}'.format(span)])/2
                    - df['PF_L{}'.format(span)] - df['PF_opp_L{}'.format(span)] - df['TOV_L{}'.format(span)] - df['TOV_opp_L{}'.format(span)]))
        
    return df
  
  
  
team_stats_ewa = add_percentage_features(team_stats_ewa, span=10)

In [267]:
def add_rest_days(df: pd.DataFrame, max_rest_days: int = 7) -> pd.DataFrame:
    """
    Calculate the number of rest days between games for each team.

    Args:
    df (pd.DataFrame): Input DataFrame containing game data.
    max_rest_days (int): Maximum number of rest days to consider. Default is 8.

    Returns:
    pd.DataFrame: DataFrame with an additional 'REST' column indicating the number of rest days.
    """
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()

    # Ensure 'GAME_DATE' is in datetime format
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

    # Calculate rest days
    df['REST'] = df.groupby(['SEASON', 'TEAM_ABBREVIATION'])['GAME_DATE'].diff().dt.days

    # Cap rest days at the specified maximum
    df['REST'] = df['REST'].clip(upper=max_rest_days)

    # Fill NaN values (first game of the season) with max_rest_days
    df['REST'] = df['REST'].fillna(max_rest_days)

    return df


team_stats_ewa = add_rest_days(team_stats_ewa)

In [268]:
team_stats_ewa

,SEASON,TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_GAME,TEAM_SCORE,POINT_DIFF,WL,FGM_L10,FGA_L10,FG3M_L10,FG3A_L10,FTM_L10,FTA_L10,OREB_L10,DREB_L10,REB_L10,AST_L10,STL_L10,BLK_L10,TOV_L10,PF_L10,PTS_L10,PLUS_MINUS_L10,E_OFF_RATING_L10,E_DEF_RATING_L10,E_NET_RATING_L10,E_PACE_L10,contestedShots_L10,contestedShots2pt_L10,contestedShots3pt_L10,deflections_L10,chargesDrawn_L10,screenAssists_L10,screenAssistPoints_L10,looseBallsRecoveredOffensive_L10,looseBallsRecoveredDefensive_L10,looseBallsRecoveredTotal_L10,offensiveBoxOuts_L10,defensiveBoxOuts_L10,boxOutPlayerTeamRebounds_L10,boxOutPlayerRebounds_L10,boxOuts_L10,DIST_L10,ORBC_L10,DRBC_L10,RBC_L10,TCHS_L10,SAST_L10,FTAST_L10,PASS_L10,CFGM_L10,CFGA_L10,UFGM_L10,UFGA_L10,DFGM_L10,DFGA_L10,PTS_OFF_TOV_L10,PTS_2ND_CHANCE_L10,PTS_FB_L10,PTS_PAINT_L10,FG2M_L10,FG2A_L10,PTS_2PT_MR_L10,AST_2PM_L10,AST_3PM_L10,UAST_2PM_L10,UAST_3PM_L10,FGM_opp_L10,FGA_opp_L10,FG3M_opp_L10,FG3A_opp_L10,FTM_opp_L10,FTA_opp_L10,OREB_opp_L10,DREB_opp_L10,REB_opp_L10,AST_opp_L10,STL_opp_L10,BLK_opp_L10,TOV_opp_L10,PF_opp_L10,PTS_opp_L10,PLUS_MINUS_opp_L10,E_OFF_RATING_opp_L10,E_DEF_RATING_opp_L10,E_NET_RATING_opp_L10,E_PACE_opp_L10,contestedShots_opp_L10,contestedShots2pt_opp_L10,contestedShots3pt_opp_L10,deflections_opp_L10,chargesDrawn_opp_L10,screenAssists_opp_L10,screenAssistPoints_opp_L10,looseBallsRecoveredOffensive_opp_L10,looseBallsRecoveredDefensive_opp_L10,looseBallsRecoveredTotal_opp_L10,offensiveBoxOuts_opp_L10,defensiveBoxOuts_opp_L10,boxOutPlayerTeamRebounds_opp_L10,boxOutPlayerRebounds_opp_L10,boxOuts_opp_L10,DIST_opp_L10,ORBC_opp_L10,DRBC_opp_L10,RBC_opp_L10,TCHS_opp_L10,SAST_opp_L10,FTAST_opp_L10,PASS_opp_L10,CFGM_opp_L10,CFGA_opp_L10,UFGM_opp_L10,UFGA_opp_L10,DFGM_opp_L10,DFGA_opp_L10,PTS_OFF_TOV_opp_L10,PTS_2ND_CHANCE_opp_L10,PTS_FB_opp_L10,PTS_PAINT_opp_L10,FG2M_opp_L10,FG2A_opp_L10,PTS_2PT_MR_opp_L10,AST_2PM_opp_L10,AST_3PM_opp_L10,UAST_2PM_opp_L10,UAST_3PM_opp_L10,WIN_PCT_L10_L10,OREB_PCT_L10,OREB_PCT_opp_L10,DREB_PCT_L10,DREB_PCT_opp_L10,REB_PCT_L10,REB_PCT_opp_L10,TS_PCT_L10,TS_PCT_opp_L10,EFG_PCT_L10,EFG_PCT_opp_L10,AST_RATIO_L10,AST_RATIO_opp_L10,TOV_PCT_L10,TOV_PCT_opp_L10,PIE_L10,REST,FG2_PCT_L10,FG3_PCT_L10
0,2016-17,ATL,2016-10-27,0021600014,ATL vs. WAS,1,114,15,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN
1,2016-17,ATL,2016-10-29,0021600026,ATL @ PHI,0,104,32,1,41.540785,83.081571,11.329305,24.546828,13.217523,16.993958,13.217523,35.876133,49.093656,26.435045,12.273414,6.608761,19.826284,17.938066,107.628399,14.161631,110.800000,90.900000,19.900000,105.920000,67.975831,50.981873,16.993958,19.826284,0.000000,6.608761,14.161631,0.000000,0.000000,5.664653,0.000000,0.000000,0.000000,0.000000,0.000000,16.399169,25.490937,66.087613,90.634441,407.854985,2.832326,0.944109,283.232628,19.826284,41.540785,21.714502,41.540785,11.329305,20.770393,24.546828,18.882175,15.105740,41.540785,30.211480,58.534743,17.938066,15.105740,11.329305,15.105740,0.000000,37.764350,88.746224,5.664653,18.882175,12.273414,16.993958,11.329305,26.435045,37.764350,23.602719,13.217523,3.776435,17.938066,18.882175,93.466767,-14.161631,90.900000,110.800000,-19.900000,105.920000,50.037764,33.043807,16.993958,23.602719,0.000000,7.552870,16.049849,0.000000,0.000000,4.720544,0.000000,0.000000,0.000000,0.000000,0.000000,16.276435,36.820242,44.373112,77.416918,402.190332,4.720544,0.000000,279.456193,19.826284,41.540785,17.938066,47.205438,14.161631,21.714502,23.602719,5.664653,16.993958,45.317221,32.099698,69.864

In [269]:
def make_matchups_2(df):
    
    home_teams = df.loc[df['HOME_GAME'] == 1]
    home_teams = home_teams.add_prefix('HOME_')

    away_teams = df.loc[df['HOME_GAME'] == 0]
    away_teams = away_teams.add_prefix('AWAY_')

    drop_cols = ['AWAY_SEASON', 'AWAY_GAME_DATE',
                            'AWAY_MATCHUP', 'AWAY_HOME_GAME', 'AWAY_POINT_DIFF', 'AWAY_WL',
                            # 'AWAY_ATS_DIFF', 'AWAY_SPREAD', 'AWAY_TEAM_COVERED'
                            ]
    
    away_teams = away_teams.drop(columns=drop_cols)

    full_matchup_ewa = pd.merge(home_teams, away_teams, how = 'inner',
                                left_on = 'HOME_GAME_ID', right_on = 'AWAY_GAME_ID') 
    
    full_matchup_ewa = full_matchup_ewa.rename(columns = {'HOME_SEASON':'SEASON',
                                                          'HOME_GAME_DATE':'GAME_DATE',
                                                          'HOME_GAME_ID':'GAME_ID',
                                                          'HOME_MATCHUP':'MATCHUP'})
    
    full_matchup_ewa = full_matchup_ewa.drop(columns=['AWAY_GAME_ID'])
    
    return full_matchup_ewa


start = time.time()
full_matchups_team_stats_ewa = make_matchups_2(team_stats_ewa)
end = time.time()
print(f"Time taken: {end - start} seconds")


Time taken: 0.18192505836486816 seconds


In [270]:
cols = [x for x in full_matchups_team_stats_ewa.columns if 'RB' in x or 'Rebound' in x]

full_matchups_team_stats_ewa[cols]

full_matchups_team_stats_ewa

,SEASON,HOME_TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_HOME_GAME,HOME_TEAM_SCORE,HOME_POINT_DIFF,HOME_WL,HOME_FGM_L10,HOME_FGA_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_REB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_PTS_L10,HOME_PLUS_MINUS_L10,HOME_E_OFF_RATING_L10,HOME_E_DEF_RATING_L10,HOME_E_NET_RATING_L10,HOME_E_PACE_L10,HOME_contestedShots_L10,HOME_contestedShots2pt_L10,HOME_contestedShots3pt_L10,HOME_deflections_L10,HOME_chargesDrawn_L10,HOME_screenAssists_L10,HOME_screenAssistPoints_L10,HOME_looseBallsRecoveredOffensive_L10,HOME_looseBallsRecoveredDefensive_L10,HOME_looseBallsRecoveredTotal_L10,HOME_offensiveBoxOuts_L10,HOME_defensiveBoxOuts_L10,HOME_boxOutPlayerTeamRebounds_L10,HOME_boxOutPlayerRebounds_L10,HOME_boxOuts_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_2ND_CHANCE_L10,HOME_PTS_FB_L10,HOME_PTS_PAINT_L10,HOME_FG2M_L10,HOME_FG2A_L10,HOME_PTS_2PT_MR_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_FGM_opp_L10,HOME_FGA_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_REB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_PTS_opp_L10,HOME_PLUS_MINUS_opp_L10,HOME_E_OFF_RATING_opp_L10,HOME_E_DEF_RATING_opp_L10,HOME_E_NET_RATING_opp_L10,HOME_E_PACE_opp_L10,HOME_contestedShots_opp_L10,HOME_contestedShots2pt_opp_L10,HOME_contestedShots3pt_opp_L10,HOME_deflections_opp_L10,HOME_chargesDrawn_opp_L10,HOME_screenAssists_opp_L10,HOME_screenAssistPoints_opp_L10,HOME_looseBallsRecoveredOffensive_opp_L10,HOME_looseBallsRecoveredDefensive_opp_L10,HOME_looseBallsRecoveredTotal_opp_L10,HOME_offensiveBoxOuts_opp_L10,...,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_OFF_TOV_L10,AWAY_PTS_2ND_CHANCE_L10,AWAY_PTS_FB_L10,AWAY_PTS_PAINT_L10,AWAY_FG2M_L10,AWAY_FG2A_L10,AWAY_PTS_2PT_MR_L10,AWAY_AST_2PM_L10,AWAY_AST_3PM_L10,AWAY_UAST_2PM_L10,AWAY_UAST_3PM_L10,AWAY_FGM_opp_L10,AWAY_FGA_opp_L10,AWAY_FG3M_opp_L10,AWAY_FG3A_opp_L10,AWAY_FTM_opp_L10,AWAY_FTA_opp_L10,AWAY_OREB_opp_L10,AWAY_DREB_opp_L10,AWAY_REB_opp_L10,AWAY_AST_opp_L10,AWAY_STL_opp_L10,AWAY_BLK_opp_L10,AWAY_TOV_opp_L10,AWAY_PF_opp_L10,AWAY_PTS_opp_L10,AWAY_PLUS_MINUS_opp_L10,AWAY_E_OFF_RATING_opp_L10,AWAY_E_DEF_RATING_opp_L10,AWAY_E_NET_RATING_opp_L10,AWAY_E_PACE_opp_L10,AWAY_contestedShots_opp_L10,AWAY_contestedShots2pt_opp_L10,AWAY_contestedShots3pt_opp_L10,AWAY_deflections_opp_L10,AWAY_chargesDrawn_opp_L10,AWAY_screenAssists_opp_L10,AWAY_screenAssistPoints_opp_L10,AWAY_looseBallsRecoveredOffensive_opp_L10,AWAY_looseBallsRecoveredDefensive_opp_L10,AWAY_looseBallsRecoveredTotal_opp_L10,AWAY_offensiveBoxOuts_opp_L10,AWAY_defensiveBoxOuts_opp_L10,AWAY_boxOutPlayerTeamRebounds_opp_L10,AWAY_boxOutPlayerRebounds_opp_L10,AWAY_boxOuts_opp_L10,AWAY_DIST_opp_L10,AWAY_ORBC_opp_L10,AWAY_DRBC_opp_L10,AWAY_RBC_opp_L10,AWAY_TCHS_opp_L10,AWAY_SAST_opp_L10,AWAY_FTAST_opp_L10,AWAY_PASS_opp_L10,AWAY_CFGM_opp_L10,AWAY_CFGA_opp_L10,AWAY_UFGM_opp_L10,AWAY_UFGA_opp_L10,AWAY_DFGM_opp_L10,AWAY_DFGA_opp_L10,AWAY_PTS_OFF_TOV_opp_L10,AWAY_PTS_2ND_CHANCE_opp_L10,AWAY_PTS_FB_opp_L10,AWAY_PTS_PAINT_opp_L10,AWAY_FG2M_opp_L10,AWAY_FG2A_opp_L10,AWAY_PTS_2PT_MR_opp_L10,AWAY_AST_2PM_opp_L10,AWAY_AST_3PM_opp_L10,AWAY_UAST_2PM_opp_L10,AWAY_UAST_3PM_opp_L10,AWAY_WIN_PCT_L10_L10,AWAY_OREB_PCT_L10,AWAY_OREB_PCT_opp_L10,AWAY_DREB_PCT_L10,AWAY_DREB_PCT_opp_L10,AWAY_REB_PCT_L10,AWAY_REB_PCT_opp_L10,AWAY_TS_PCT_L10,AWAY_TS_PCT_opp_L10,AWAY_EFG_PCT_L10,AWAY_EFG_PCT_opp_L10,AWAY_AST_RATIO_L10,AWAY_AST_RATIO_opp_L10,AWAY_TOV_PCT_L10,AWAY_TOV_PCT_opp_L10,AWAY_PIE_L10,AWAY_REST,AWAY_FG2_PCT_L10,AWAY_FG3_PCT_L10
0,201

In [272]:
full_matchups_team_stats_ewa[['HOME_FG2M_L10' ,'HOME_FG2A_L10', 'HOME_FG2_PCT_L10']]

,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG2_PCT_L10
0,NaN,NaN,NaN
1,33.573568,61.437827,0.546464
2,29.287646,57.254366,0.511536
3,28.131515,55.147263,0.510116
4,29.982604,55.722928,0.538066
...,...,...,...
10203,28.611159,53.632790,0.533464
10204,28.479060,52.622632,0.541194
10205,29.837559,52.515619,0.568165
10206,28.906407,51.122845,0.565430


In [252]:
def engineer_features(df, keep_original=False):
    
    df = df.copy()
    # List of features to create relative metrics for
    relative_features = [col[5:] for col in df.columns if col.startswith('HOME_') and col.endswith('_L10')]
    relative_features = list(set(relative_features) - set(['HOME_GAME', 'TEAM_SCORE', 'POINT_DIFF', 'WL']))

    # Create relative metrics
    for feature in relative_features:
        df[f'REL_{feature}'] = df[f'HOME_{feature}'] - df[f'AWAY_{feature}']


    # Matchup-specific metrics
    df['HOME_OFF_VS_DEF'] = df['HOME_E_OFF_RATING_L10'] - df['AWAY_E_DEF_RATING_L10']
    df['AWAY_OFF_VS_DEF'] = df['AWAY_E_OFF_RATING_L10'] - df['HOME_E_DEF_RATING_L10']

    df['HOME_OFF_VS_DEF'] = df['HOME_E_OFF_RATING_L10'] - df['AWAY_E_DEF_RATING_L10']
    df['AWAY_OFF_VS_DEF'] = df['AWAY_E_OFF_RATING_L10'] - df['HOME_E_DEF_RATING_L10']

    df['HOME_OREB_OPPORTUNITY_RATIO'] = df['HOME_ORBC_L10'] / df['AWAY_DRBC_L10']
    df['AWAY_OREB_OPPORTUNITY_RATIO'] = df['AWAY_ORBC_L10'] / df['HOME_DRBC_L10']
    # Rest advantage
    df['REST_ADVANTAGE'] = df['HOME_REST'] - df['AWAY_REST']

    df['HOME_2ND_CHANCE_EFFICIENCY'] = df['HOME_PTS_2ND_CHANCE_L10'] / df['HOME_ORBC_L10']
    df['AWAY_2ND_CHANCE_EFFICIENCY'] = df['AWAY_PTS_2ND_CHANCE_L10'] / df['AWAY_ORBC_L10']
    

    # Creating interaction terms
    interaction_features = ['E_OFF_RATING_L10', 'E_DEF_RATING_L10', 'TS_PCT_L10', 'EFG_PCT_L10', 'AST_RATIO_L10', 'TOV_PCT_L10', 'REB_PCT_L10']
    for feature in interaction_features:
        df[f'HOME_{feature}_INTERACTION'] = df[f'HOME_{feature}'] * df[f'HOME_{feature}_opp'] if f'HOME_{feature}_opp' in df.columns else df[f'HOME_{feature}']
        df[f'AWAY_{feature}_INTERACTION'] = df[f'AWAY_{feature}'] * df[f'AWAY_{feature}_opp'] if f'AWAY_{feature}_opp' in df.columns else df[f'AWAY_{feature}']


    df['HOME_ORBC_TS_INTERACTION'] = df['HOME_ORBC_L10'] * df['HOME_TS_PCT_L10']
    df['AWAY_ORBC_TS_INTERACTION'] = df['AWAY_ORBC_L10'] * df['AWAY_TS_PCT_L10']
    df['HOME_DRBC_OPP_TS_INTERACTION'] = df['HOME_DRBC_L10'] * df['AWAY_TS_PCT_L10']
    df['AWAY_DRBC_OPP_TS_INTERACTION'] = df['AWAY_DRBC_L10'] * df['HOME_TS_PCT_L10']
    

    teams = ['HOME', 'AWAY']
    for team in teams:
        # Rebounding Efficiency Ratios
        df[f'{team}_OREB_EFFICIENCY'] = df[f'{team}_OREB_L10'] / df[f'{team}_ORBC_L10']
        df[f'{team}_DREB_EFFICIENCY'] = df[f'{team}_DREB_L10'] / df[f'{team}_DRBC_L10']
        df[f'{team}_OVERALL_REB_EFFICIENCY'] = (df[f'{team}_OREB_L10'] + df[f'{team}_DREB_L10']) / df[f'{team}_RBC_L10']
        
        # Box Out Effectiveness
        df[f'{team}_BOX_OUT_TO_CHANCE_RATIO'] = (df[f'{team}_boxOutPlayerRebounds_L10'] + df[f'{team}_boxOutPlayerTeamRebounds_L10']) / df[f'{team}_RBC_L10']
        
        # Rebounding Opportunity Ratios
        df[f'{team}_OREB_CHANCE_RATIO'] = df[f'{team}_ORBC_L10'] / df[f'{team}_RBC_L10']
        df[f'{team}_DREB_CHANCE_RATIO'] = df[f'{team}_DRBC_L10'] / df[f'{team}_RBC_L10']
        
        # Rebounding Conversion Rates
        df[f'{team}_OREB_CONVERSION_RATE'] = df[f'{team}_OREB_L10'] / df[f'{team}_ORBC_L10']
        df[f'{team}_DREB_CONVERSION_RATE'] = df[f'{team}_DREB_L10'] / df[f'{team}_DRBC_L10']
        
        # Rebounding Chance Differentials
        df[f'{team}_NET_REB_CHANCES'] = df[f'{team}_RBC_L10'] - df[f'{team}_RBC_opp_L10']
        df[f'{team}_NET_OREB_CHANCES'] = df[f'{team}_ORBC_L10'] - df[f'{team}_ORBC_opp_L10']
        df[f'{team}_NET_DREB_CHANCES'] = df[f'{team}_DRBC_L10'] - df[f'{team}_DRBC_opp_L10']
        
        # Box Out Impact
        df[f'{team}_BOX_OUT_EFFECTIVENESS'] = (df[f'{team}_boxOutPlayerRebounds_L10'] + df[f'{team}_boxOutPlayerTeamRebounds_L10']) / df[f'{team}_RBC_L10']

    
    # df = pd.get_dummies(df, columns=['HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION'])

    if not keep_original:
        columns_to_drop = [col for col in df.columns if col.startswith('HOME_') or col.startswith('AWAY_')]
        columns_to_keep = [col for col in df.columns if col.startswith('HOME_TEAM_ABBREVIATION') or col.startswith('AWAY_TEAM_ABBREVIATION')]
        columns_to_keep += ['HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE', 'HOME_POINT_DIFF', 'HOME_WL']
        columns_to_drop = list(set(columns_to_drop) - set(columns_to_keep))
        df = df.drop(columns=columns_to_drop)

    high_corr_pairs = find_high_correlations(df, threshold=0.7)
    # Dropping highly correlated features
    drop_cols = [pair[0] for pair in high_corr_pairs]
    df = df.drop(columns=drop_cols)


    return df


final_df = engineer_features(full_matchups_team_stats_ewa)

final_df

C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_65804\2405423281.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'REL_{feature}'] = df[f'HOME_{feature}'] - df[f'AWAY_{feature}']
C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_65804\2405423281.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'REL_{feature}'] = df[f'HOME_{feature}'] - df[f'AWAY_{feature}']
C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_65804\2405423281.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

,SEASON,HOME_TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_TEAM_SCORE,HOME_POINT_DIFF,AWAY_TEAM_ABBREVIATION,AWAY_TEAM_SCORE,REL_FG3A_opp_L10,REL_chargesDrawn_opp_L10,REL_boxOuts_L10,REL_UFGA_L10,REL_E_NET_RATING_opp_L10,REL_FGA_opp_L10,REL_BLK_L10,REL_SAST_L10,REL_DFGA_L10,REL_chargesDrawn_L10,REL_PTS_OFF_TOV_opp_L10,REL_screenAssistPoints_opp_L10,REL_DIST_opp_L10,REL_deflections_L10,REL_contestedShots2pt_L10,REL_PTS_2ND_CHANCE_opp_L10,REL_defensiveBoxOuts_opp_L10,REL_FTM_L10,REL_OREB_L10,REL_DREB_opp_L10,REL_looseBallsRecoveredDefensive_L10,REL_RBC_opp_L10,REL_FGA_L10,REL_DREB_PCT_L10,REL_FTM_opp_L10,REL_PTS_FB_opp_L10,REL_DREB_L10,REL_AST_2PM_opp_L10,REL_FTAST_L10,REL_boxOutPlayerRebounds_opp_L10,REL_REB_PCT_opp_L10,REL_CFGM_opp_L10,REL_AST_RATIO_L10,REL_FG2A_L10,REL_looseBallsRecoveredTotal_opp_L10,REL_TCHS_L10,REL_offensiveBoxOuts_L10,REL_boxOutPlayerRebounds_L10,REL_CFGA_L10,REL_STL_opp_L10,REL_screenAssistPoints_L10,REL_PTS_FB_L10,REL_UFGA_opp_L10,REL_DFGM_opp_L10,REL_looseBallsRecoveredDefensive_opp_L10,REL_UAST_3PM_L10,REL_UFGM_opp_L10,REL_PTS_2PT_MR_opp_L10,REL_BLK_opp_L10,REL_SAST_opp_L10,REL_FTAST_opp_L10,REL_RBC_L10,REL_looseBallsRecoveredOffensive_L10,REL_TCHS_opp_L10,REL_contestedShots_opp_L10,REL_PTS_2PT_MR_L10,REL_looseBallsRecoveredTotal_L10,REL_E_PACE_L10,REL_deflections_opp_L10,REL_UAST_2PM_L10,REL_UAST_2PM_opp_L10,REL_looseBallsRecoveredOffensive_opp_L10,REL_offensiveBoxOuts_opp_L10,REL_UAST_3PM_opp_L10,REST_ADVANTAGE
0,2016-17,ATL,2016-10-27,0021600014,ATL vs. WAS,114,15,WAS,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2016-17,ATL,2016-10-31,0021600044,ATL vs. SAC,106,11,SAC,95,-1.780474,-0.620167,0.000000,3.376627,-29.814751,4.847807,3.265719,1.646565,-0.896251,-1.049177,-4.940094,1.652732,0.081117,-2.081138,8.250117,-2.513122,0.000000,-10.367962,-0.307124,-0.790764,0.000000,0.605462,-0.697911,0.075434,-11.169586,-1.750502,9.486928,-2.112868,0.572165,0.000000,-0.068597,-4.322066,2.081470,1.212844,-2.611514,-43.053833,0.000000,0.000000,-3.028184,3.134148,0.908426,2.291008,5.037057,3.303282,0.000000,-0.697584,-1.311872,-10.554726,-1.336251,0.300082,-2.599149,25.618207,0.000000,6.265011,-5.617332,3.589799,-2.380645,8.071120,-3.638333,-0.166244,-3.301722,0.000000,0.000000,-0.045225,0.0
2,2016-17,ATL,2016-11-02,0021600059,ATL vs. LAL,116,-7,LAL,123,-5.194633,-0.876078,0.000000,5.970468,-29.405831,-0.930461,3.149246,3.287987,-7.824669,0.741598,-7.542655,9.908386,0.890622,5.116328,6.556475,-9.578658,0.000000,2.352849,2.115681,0.614113,0.000000,1.588706,-0.410920,0.093486,-7.898142,-2.883476,4.125041,0.455743,0.717625,0.000000,-0.055084,-3.237382,11.302087,1.737153,-0.185618,31.147942,0.000000,0.000000,-6.074575,1.260857,0.036600,1.750752,0.760707,1.492548,0.000000,-1.157980,-0.820218,3.035786,-0.551054,0.920564,-0.387363,14.819947,0.000000,33.742595,-1.528691,2.122882,-1.236450,-4.464489,4.310560,-6.573553,-4.250359,0.000000,0.000000,-0.069463,1.0
3,2016-17,ATL,2016-11-05,0021600084,ATL vs. HOU,112,15,HOU,97,-8.402451,-1.454740,0.000000,-6.188182,-2.553562,-4.232303,0.383559,1.129985,0.231970,-0.273526,0.978371,6.152328,-0.112854,3.999616,0.755642,-3.472423,0.000000,2.618497,-0.798768,1.922307,0.000000,1.221270,-3.507336,0.022042,-2.377639,2.600380,0.431006,4.177477,1.319899,0.000000,0.007531,1.553974,0.392413,7.059091,-0.922550,32.643642,0.000000,0.000000,2.680846,1.091430,-6.214719,-0.991221,-7.961212,-1.661152,0.000000,-2.972049,-4.348432,4.543245,1.729237,-0.324071,-0.543885,-1.038928,0.000000,-41.483863,-4.355578,6.168762,-1.954104,3.501102,-0.457724,-1.299462,-2.716511,0.000000,0.000000,-1.032943,-2.0
4,2016-17,ATL,2016-11-09,0021600111,ATL vs. CHI,115,8,CHI,107,0.782766,-0.954468,0.000000,1.907101,-2.681690,-4.767225,-0.796662,1.032031,-1.623559,0.640678,1.282853,0.694900,-0.982443,3.3

In [279]:
full_matchups_team_stats_ewa[[x for x in full_matchups_team_stats_ewa.columns if 'WIN_PCT' in x]]

full_matchups_team_stats_ewa[['HOME_PTS_L10', 'HOME_E_OFF_RATING_L10']]

print(full_matchups_team_stats_ewa.shape)
high_corr_pairs = find_high_correlations(full_matchups_team_stats_ewa, threshold=0.7)
# Dropping highly correlated features
drop_cols = [pair[0] for pair in high_corr_pairs]
final_df = full_matchups_team_stats_ewa.drop(columns=drop_cols)


print(final_df.shape)

(10208, 289)
(10208, 127)


In [280]:
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    model = xgb.XGBRegressor(random_state=42, n_jobs=-1)  # n_jobs=-1 uses all available cores
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    
    return model, mse, mae, r2


def season_to_string(s):
    return str(s) + "-" + str(s+1)[-2:]

def time_series_cv(df, start_season=2016, end_season=2023):
    results = []
    
    for test_season in range(start_season + 2, end_season + 1):
        
        print(f"Training on seasons {start_season}-{test_season-1}, testing on season {test_season}")
        
        # Split data
        train_data = df[df['SEASON'] < season_to_string(test_season)]
        test_data = df[df['SEASON'] == season_to_string(test_season)]
        
        # Prepare features and target
        features = [col for col in df.columns if col not in ['SEASON', 'GAME_DATE', 'GAME_ID', 'HOME_WL', 'MATCHUP', 'HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE', 'HOME_POINT_DIFF'
                                                             , 'HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION'
                                                             ]]
        X_train = train_data[features].astype(float)
        y_train_home = train_data['HOME_TEAM_SCORE']
        y_train_away = train_data['AWAY_TEAM_SCORE']
        
        X_test = test_data[features].astype(float)
        y_test_home = test_data['HOME_TEAM_SCORE']
        y_test_away = test_data['AWAY_TEAM_SCORE']
        
        # Train and evaluate home score model
        home_model, home_mse, home_mae, home_r2 = train_and_evaluate_model(X_train, y_train_home, X_test, y_test_home)
        
        # Train and evaluate away score model
        away_model, away_mse, away_mae, away_r2 = train_and_evaluate_model(X_train, y_train_away, X_test, y_test_away)
        
        results.append({
            'Test Season': test_season,
            'Home MSE': home_mse,
            'Home MAE': home_mae,
            'Home R2': home_r2,
            'Away MSE': away_mse,
            'Away MAE': away_mae,
            'Away R2': away_r2
        })
        
        # Optionally, save feature importance
        feature_importance = pd.DataFrame({
            'feature': features,
            'importance_home': home_model.feature_importances_,
            'importance_away': away_model.feature_importances_
        })
        feature_importance.to_csv(f'feature_importance_season_{test_season}.csv', index=False)
    
    return pd.DataFrame(results)

# Main execution
# df = pd.read_csv('your_data.csv')  # Load your data
# df = prepare_data(df)
# df = engineer_features(df)

results = time_series_cv(final_df)
print(results)

# Optionally, you can save the results
# results.to_csv('model_performance_by_season.csv', index=False)

Training on seasons 2016-2017, testing on season 2018
Training on seasons 2016-2018, testing on season 2019
Training on seasons 2016-2019, testing on season 2020
Training on seasons 2016-2020, testing on season 2021
Training on seasons 2016-2021, testing on season 2022
Training on seasons 2016-2022, testing on season 2023
   Test Season    Home MSE   Home MAE   Home R2    Away MSE   Away MAE  \
0         2018  188.655940  10.684888 -0.179263  197.187114  11.140750   
1         2019  180.804318  10.650202 -0.136876  172.649366  10.465782   
2         2020  183.521204  10.704563 -0.173522  177.000815  10.429149   
3         2021  162.563262  10.143618 -0.053239  167.877496  10.284065   
4         2022  169.407775  10.338518 -0.179088  163.421309  10.065522   
5         2023  164.931441  10.328378  0.006049  162.709375  10.158811   

    Away R2  
0 -0.258666  
1 -0.160861  
2 -0.122366  
3 -0.037349  
4 -0.135367  
5  0.024271  


In [277]:
fi_2023 = pd.read_csv('feature_importance_season_2023.csv')
fi_2023.sort_values(['importance_home'], ascending=False).head(30)

,feature,importance_home,importance_away
17,HOME_E_OFF_RATING_L10,0.051820,0.011681
159,AWAY_E_PACE_L10,0.027823,0.018855
260,AWAY_WIN_PCT_L10_L10,0.019965,0.021255
121,HOME_WIN_PCT_L10_L10,0.018823,0.018782
20,HOME_E_PACE_L10,0.014518,0.019199
125,HOME_DREB_PCT_opp_L10,0.012451,0.000762
234,AWAY_boxOuts_opp_L10,0.008876,0.000000
90,HOME_looseBallsRecoveredTotal_opp_L10,0.007721,0.003149
18,HOME_E_DEF_RATING_L10,0.007453,0.027698
263,AWAY_DREB_PCT_L10,0.007043,0.008359


In [281]:
pd.options.display.max_rows = 200
fi_2023 = pd.read_csv('feature_importance_season_2023.csv')
fi_2023.sort_values(['importance_home'], ascending=False)

,feature,importance_home,importance_away
74,AWAY_E_PACE_L10,0.053607,0.036188
73,AWAY_E_DEF_RATING_L10,0.032104,0.008495
14,HOME_E_PACE_L10,0.024326,0.036574
48,HOME_DRBC_opp_L10,0.019031,0.007658
24,HOME_TCHS_L10,0.017849,0.009797
86,AWAY_DFGM_L10,0.014097,0.007208
31,HOME_PTS_FB_L10,0.012316,0.006305
89,AWAY_PTS_PAINT_L10,0.011750,0.005770
96,AWAY_OREB_opp_L10,0.011572,0.008540
12,HOME_PLUS_MINUS_L10,0.011060,0.006430


In [ ]:
def create_interactions(df, features_to_interact):
    for i, feature1 in enumerate(features_to_interact):
        for feature2 in features_to_interact[i+1:]:
            df[f'{feature1}_{feature2}_interaction'] = df[feature1] * df[feature2]
    return df

def train_and_evaluate(X, y, interactions=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    if interactions:
        X_train = create_interactions(X_train, interactions)
        X_test = create_interactions(X_test, interactions)
    
    model = xgb.XGBRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return mse, feature_importance

# Assume df is your dataframe and 'target' is your target variable
# df = pd.read_csv('your_data.csv')
# X = df.drop('target', axis=1)
# y = df['target']

# Train model without interactions
mse_without, importance_without = train_and_evaluate(X, y)

print("MSE without interactions:", mse_without)
print("\nTop 10 features without interactions:")
print(importance_without.head(10))

# Select top features for interactions
top_features = importance_without['feature'].head(5).tolist()

# Train model with interactions
mse_with, importance_with = train_and_evaluate(X, y, interactions=top_features)

print("\nMSE with interactions:", mse_with)
print("\nTop 10 features with interactions:")
print(importance_with.head(10))

print(f"\nMSE Improvement: {mse_without - mse_with}")